In [27]:
%pip install jinja2
%pip install python-dotenv
%pip install requests
%pip install wonderwords

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [28]:
import os
import random
import requests

from dotenv import load_dotenv
from IPython.display import display, Markdown
from jinja2 import Template
from wonderwords import RandomWord

In [29]:
# SPOTIFY API CONSTANTS

load_dotenv()

CLIENT_ID = os.environ.get('CLIENT_ID')
CLIENT_SECRET = os.environ.get('CLIENT_SECRET')
SPOTIFY_BASE_URL = 'https://api.spotify.com/v1/'

In [30]:
# SPOTIFY API AUTHENTICATION

def get_spotify_token(client_id, client_secret):
    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {
        "grant_type": "client_credentials",
        "client_id": client_id,
        "client_secret": client_secret
    }
    
    response = requests.post(url, headers=headers, data=data)
    response_data = response.json()
    
    if response.status_code == 200:
        return response_data['access_token']
    else:
        raise Exception(f"Error fetching token: {response_data}")

TOKEN = get_spotify_token(CLIENT_ID, CLIENT_SECRET)
HEADERS = {
        "Authorization": f"Bearer {TOKEN}"
    }
print(TOKEN)

BQBkZevvF4YU9HU6MDNr9gYqPMarNzLa8FVSG-_T_cTOpz6MgusNjIUI869gvFO4heXkRcuPNDJDAeHTiBh2urep5lT_QWRvMWzIf0xHRtTBT8Ft8Ug


In [31]:
# SPOTIFY API FUNCTIONS

r = RandomWord()
def get_random_album_id():
    random_word = r.word()
    
    search_url = SPOTIFY_BASE_URL + "search"
    params = {
        "q": random_word,
        "type": "album",
        "tag": "hipster", # to get more obscure results
        "limit": 50
    }
    response = requests.get(search_url, headers=HEADERS, params=params)
    response_data = response.json()
    
    albums = response_data.get('albums', {}).get('items', [])
    non_single_albums = [album for album in albums if album['album_type'] == 'album']
    
    if non_single_albums:
        random_album = random.choice(non_single_albums)
        return random_album['id']
    else:
        return None
    
def get_album_info(album_id):
    url = SPOTIFY_BASE_URL + f"albums/{album_id}"
    response = requests.get(url, headers=HEADERS)
    album = response.json()
    
    album_info = {
        'name': album['name'],
        'id': album['id'],
        'date': album['release_date'],
        'artists': [a['name'] for a in album['artists']],
        'cover': album['images'][0]['url'],
        'genres': album['genres'], # most often not yet classified
        'label': album['label']
    }

    return album_info

def get_album_tracks(album_id):
    url = SPOTIFY_BASE_URL + f"albums/{album_id}/tracks"
    response = requests.get(url, headers=HEADERS)
    response_data = response.json()

    tracks = {}
    for t in response_data.get('items', []):
        track = {
            'name': t['name'],
            'artists': [a['name'] for a in t['artists']]
        }
        tracks[t['track_number']] = track

    return tracks

def get_random_album_information():
    album_id = get_random_album_id()
    album = get_album_info(album_id)
    album['tracks'] = get_album_tracks(album_id)
    return album

def display_album(album):
    if album:       
        display(Markdown(f"**{album['name']}**"))
        display(Markdown(f"![Album Cover]({album['cover']})"))
        display(Markdown(f"**Artists**: {', '.join(album['artists'])}"))
        display(Markdown(f"**Release Date**: {album['date']}"))
        display(Markdown(f"**Genres**: {', '.join(album['genres'])}"))
        display(Markdown(f"**Label**: {album['label']}"))
        display(Markdown(f"**Tracks**"))
        for track_number, track in album['tracks'].items():
            display(Markdown(f"{track_number}. {track['name']} - {', '.join(track['artists'])}"))
    else:
        display(Markdown("No albums found. Try again!"))



In [32]:
# GET RANDOM ALBUM INFORMATION

album = get_random_album_information()
display_album(album)

**Music For Growing Flowers**

![Album Cover](https://i.scdn.co/image/ab67616d0000b27380d4bd12daae51e871f5d79a)

**Artists**: Erland Cooper

**Release Date**: 2022-08-19

**Genres**: 

...

4. Music For Growing Flowers - Pt. 4 - Erland Cooper

5. Music For Growing Flowers - Pt. 5 - Erland Cooper

6. Music For Growing Flowers - Pt. 6 - Erland Cooper

7. Music For Growing Flowers - Pt. 7 - Erland Cooper

8. Music For Growing Flowers - Pt. 8 - Erland Cooper

In [33]:
# JINJA TEMPLATE

template = Template("""\
Create a square album cover for the following album:

Album name : {{ album.name }}
Artist{% if album.artists|length > 1 %}s{% endif %} : {{ album.artists | join(', ') }}
Release Date : {{ album.date }}
Label : {{ album.label }}
{% if album.genres %}Genres : {{ album.genres | join(', ') }}{% endif %}
Tracks:
{% for track_number, track in album.tracks.items() %}{{ track_number }}. {{ track.name }} by {{ track.artists | join(', ') }}\n{% endfor %}
""")

print(template.render(album=album))

Create a square album cover for the following album:

Album name : Music For Growing Flowers
Artist : Erland Cooper
Release Date : 2022-08-19
Label : Mercury Classics

Tracks:
1. Music For Growing Flowers - Pt. 1 by Erland Cooper
2. Music For Growing Flowers - Pt. 2 by Erland Cooper
3. Music For Growing Flowers - Pt. 3 by Erland Cooper
4. Music For Growing Flowers - Pt. 4 by Erland Cooper
5. Music For Growing Flowers - Pt. 5 by Erland Cooper
6. Music For Growing Flowers - Pt. 6 by Erland Cooper
7. Music For Growing Flowers - Pt. 7 by Erland Cooper
8. Music For Growing Flowers - Pt. 8 by Erland Cooper



In [34]:
# Save the prompt to a text file
prompt = template.render(album=album)
with open(f"prompts/{album['name']}_prompt.txt", "w") as file:
    file.write(prompt)

print(f"Prompt saved to {album['name']}_prompt.txt")

Prompt saved to Music For Growing Flowers_prompt.txt
